# Isomatrix Tools

> Tools for converting isomatrix files into anndata objects for integration with the Scanpy ecosystem.


In [1]:
#| default_exp isomatrix_tools

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
def download_test_data():
    import urllib.request
    import gzip
    import shutil
    import os

    # URL of the file to be downloaded
    url = "https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748087/suppl/GSM3748087%5F190c.isoforms.matrix.txt.gz"

    # Download the file from `url` and save it locally under `file.txt.gz`:
    urllib.request.urlretrieve(url, 'file.txt.gz')

    # Check if the file is downloaded correctly
    if os.path.exists('file.txt.gz'):
        print("File downloaded successfully")
        # Now we need to extract the file
        with gzip.open('file.txt.gz', 'rb') as f_in:
            with open('sample_isomatrix.txt', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print("File extracted successfully")
        return os.path.abspath('sample_isomatrix.txt')
    else:
        print("Failed to download the file")
        return None



In [4]:
#| export
import pandas as pd
import scanpy as sc
from scanpy import AnnData
from scipy.sparse import csr_matrix
import warnings

def isomatrix_to_anndata(file_path:str,  # The path to the isomatrix csv file to be read.
                        sparse:bool=True  # Flag to determine if the output should be a sparse matrix.
) -> AnnData: # The converted isomatrix as a scanpy compatible  anndata object
    """
    This function reads a file into a pandas DataFrame, performs some transformations, 
    and then converts the DataFrame into a sparse matrix and an AnnData object.

    """
    
    # Read in the data from the file
    df = pd.read_csv(file_path, sep='\t', index_col=0)
    # Filter out rows where the transcriptId is "undef"
    df = df.loc[df["transcriptId"] != "undef"]
    
    df = df.reset_index()
    df = df.transpose()
    
    # Extract the rows with 'gene_id', 'transcript_id', 'nb_exons' from the DataFrame
    additional_info_rows = df.loc[df.index.intersection(['geneId', 'transcriptId', 'nbExons'])]
    # Drop 'gene_id', 'transcript_id', 'nb_exons' rows from the DataFrame if they exist
    df = df.drop(['geneId', 'transcriptId', 'nbExons'], errors='ignore')

    # Convert the DataFrame to a sparse matrix if the sparse flag is True
    if sparse:
        matrix = csr_matrix(df.values.astype('float32'))
    else:
        try:
            matrix = df.values.astype('float32')
        except ValueError:
            print("Error: Non-numeric data present in the DataFrame. Cannot convert to float.")
            return None
    
    # Convert the matrix to an AnnData object
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        anndata = sc.AnnData(X=matrix, obs=pd.DataFrame(index=df.index), var=pd.DataFrame(index=df.columns))
    
    # Add additional information to the AnnData object vars
    for info in ['geneId', 'transcriptId', 'nbExons']:
        if info in additional_info_rows.index:
            anndata.var[info] = additional_info_rows.loc[info, :].values
            if info == 'nbExons':
                anndata.var[info] = anndata.var[info].astype('int32')
    
    return anndata

Example Usage of isomatrix_to_anndata:

In [5]:
import longreadtools.isomatrix_tools 
test_file = download_test_data() 
anndata = isomatrix_to_anndata(test_file)

File downloaded successfully
File extracted successfully
File downloaded successfully
File extracted successfully


Lets take a look at the anndata object generated from the isomatrix.

In [6]:
anndata

AnnData object with n_obs × n_vars = 190 × 20834
    var: 'geneId', 'transcriptId'

In [7]:
#| export

def test_isomatrix_to_anndata():
    # Test with a known file
    test_file = download_test_data()
    anndata = isomatrix_to_anndata(test_file)

    # Check the type of the returned object
    assert isinstance(anndata, sc.AnnData), "The returned object is not an AnnData object."

    # Check the dimensions of the AnnData object
    assert anndata.shape == (190, 20834), "The dimensions of the AnnData object are not as expected."

    # Check the var names of the AnnData object
    assert 'geneId' in anndata.var, "The 'geneId' is not in the var of the AnnData object."
    assert 'transcriptId' in anndata.var, "The 'transcriptId' is not in the var of the AnnData object."




In [8]:
#| hide
import nbdev; nbdev.nbdev_export()